In [17]:
import torch
import numpy as np
import random

def set_seed(seed):
    random.seed(seed)

    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
import pandas as pd

df_cc = pd.read_csv('misclassified_samples/cs_cg.csv')
df_mc = pd.read_csv('misclassified_samples/ms_cg.csv')

In [5]:
pd.set_option('display.max_colwidth', None)
df_cc.sample(10)

,sentence,target_logits,concept_logits,concept_gradient,label,concept_label
260,on god america be like i hate niggers but i will let a entity from the dark realm chill upstairs,[[0.02584497 0.97415507]],[[0.8139009 0.15481107 0.2589792 ]],"[[0.07376474, 0.14568885, -0.03609989]]",1,[[1. 0. 0.]]
832,interesting do you know a lot of jewish women in real life,[[0.9244602 0.07553983]],[[0.04945392 0.8570404 0.8552614 ]],"[[0.20188831, -0.00803704, 0.01656252]]",0,[[0. 1. 1.]]
846,this is much more than being about filthy mudsharks this is about the destiny of humanity does it become a dumbed down mulatto race where the west falls apart like africa and asians dominate the future or do we ship the nigs back where they belong,[[0.00321163 0.9967884 ]],[[0.8679665 0.72623056 0.6822817 ]],"[[0.00168284, 0.13898487, -0.00380297]]",1,[[1. 0. 1.]]
1006,the hill as always twists it up it not the religious right it the religious left that run refugee placement programs and such like the jackasses settle muzzies in paterson nj,[[0.00887083 0.99112916]],[[0.08121175 0.9344502 0.25927147]],"[[0.01785006, 0.20943324, 0.00300069]]",1,[[0. 1. 0.]]
88,<user> chinaman satanist lee hsien loong malaysian criminals have been gossiping false information to the ntuc supermarket sembawang mart staff since mid <number> swift discoveries if questioned,[[0.92801183 0.07198813]],[[0.12446217 0.05253268 0.04541463]],"[[0.22226606, -0.08524344, -0.06997377]]",0,[[0. 0. 0.]]
457,pan you cheater you are no newfag you are an expert on bloated kike identification,[[0.00311579 0.99688417]],[[0.06892045 0.97842854 0.07868374]],None,1,[[0. 1. 0.]]
184,it ain t even noon yet and niggas lying,[[0.99288285 0.0071172 ]],[[0.5795693 0.0020987 0.40699673]],"[[0.24556774, 0.05697606, -0.0106516]]",0,[[0. 0. 0.]]
987,<user> ‍ i agree the catholic church loves their muzzie infiltration at any cost they give and give and give to the sand niggers at the cost of white catholics shame on them,[[0.01127249 0.98872745]],[[0.81229925 0.9615029 0.13181043]],"[[0.01617795, 0.22334181, -0.01459285]]",1,[[0. 1. 0.]]
367,got a lot of them niggers you better and it active yeah popping willies on the wall i am saying,[[0.14155416 0.8584458 ]],[[0.7979865 0.07653762 0.22520624]],"[[0.02041706, 0.24893375, -0.00640555]]",1,[[1. 0. 0.]]
558,anyway you all see these euro niggas dangerous mofuckas,[[0.66852504 0.331475 ]],[[0.7446572 0.01557087 0.61740845]],"[[0.25879902, -0.06191985, -0.01952416]]",0,[[1. 0. 1.]]


In [4]:
import ast

def clean_and_parse_gradient(example):
    if isinstance(example, str):
        # Replace multiple spaces and commas, remove invalid characters, and ensure proper list structure
        cleaned_example = example.replace(' ', ',').replace(',,', ',').replace('[,', '[').replace(',]', ']')
        try:
            return ast.literal_eval(cleaned_example)  # Parse into a Python list
        except:
            return None  # Return None if parsing fails
    return example

df_cc['concept_gradient'] = df_cc['concept_gradient'].apply(clean_and_parse_gradient)
df_mc['concept_gradient'] = df_mc['concept_gradient'].apply(clean_and_parse_gradient)
# df_cc['tcav_score2'] = df_cc['tcav_score2'].apply(clean_and_parse_gradient)
# df_mc['tcav_score2'] = df_mc['tcav_score2'].apply(clean_and_parse_gradient)
df_cc.shape, df_mc.shape

((1055, 6), (131, 6))

In [15]:
import pandas as pd
import numpy as np
from collections import defaultdict

def process_data(df, label):
    concepts = ['obscene', 'threat', 'sexual_explicit', 'insult', 'identity_attack']
    result = defaultdict(list)
    
    for i, concept in enumerate(concepts):
        scores = df[df.label == label]['tcav_score2'].apply(lambda x: x[i])
        result[concept] = np.mean(scores)
    
    return dict(result)

data = {
    'Category': ['obscene', 'threat', 'sexual_explicit', 'insult', 'identity_attack'],
    'df_cc.label==0': list(process_data(df_cc, 0).values()),
    'df_mc.label==0': list(process_data(df_mc, 0).values()),
    'df_cc.label==1': list(process_data(df_cc, 1).values()),
    'df_mc.label==1': list(process_data(df_mc, 1).values())
}

df_result = pd.DataFrame(data)
df_result.head()

,Category,df_cc.label==0,df_mc.label==0,df_cc.label==1,df_mc.label==1
0,obscene,-0.073113,0.016186,0.048393,0.065288
1,threat,-0.061446,-0.014405,0.028681,0.035683
2,sexual_explicit,-0.127778,0.048258,0.111413,0.132550
3,insult,-0.068115,-0.010854,0.079441,0.064056
4,identity_attack,-0.056969,-0.005276,0.024179,0.050601


In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict

def process_data(df, label):
    concepts = ['Race', 'Religion', 'Gender']
    result = defaultdict(list)
    counts = defaultdict(int)
    
    for i, concept in enumerate(concepts):
        scores = df[df.label == label]['concept_gradient'].apply(lambda x: x[0][i] if x is not None and len(x) > 0 else None)
        scores = scores.dropna()  
        if len(scores) > 0:
            result[concept] = np.mean(scores)
            counts[concept] = len(scores)
        else:
            result[concept] = np.nan 
            counts[concept] = 0
    
    return dict(result), dict(counts)

data = {
    'Category': ['Race', 'Religion', 'Gender'],
}

counts = {
    'Category': ['Race', 'Religion', 'Gender'],
}

for df_name, df in [('df_cc', df_cc), ('df_mc', df_mc)]:
    for label in [0, 1]:
        means, sample_counts = process_data(df, label)
        column_name = f'{df_name} normal' if label==0 else f'{df_name} Hate Speech'
        data[column_name] = list(means.values())
        counts[column_name] = list(sample_counts.values())

# Create DataFrames
df_result = pd.DataFrame(data)
df_counts = pd.DataFrame(counts)

# counts

In [9]:
 df_result.head()

,Category,df_cc normal,df_cc Hate Speech,df_mc normal,df_mc Hate Speech
0,Race,0.190880,0.055938,0.204719,0.013238
1,Religion,-0.078085,0.208046,-0.045716,0.276779
2,Gender,-0.029476,0.003654,-0.007559,0.020512


In [14]:
import numpy as np

idx2name = {
    0: 'Race', 
    1: 'Religion', 
    2: 'Gender', 
}
sentences = {}
for i, row in df_cc[df_cc.label==1].iterrows():

    try:
        indices = np.argsort(row['concept_gradient'])[0][-2:][::-1]
        # print(indices)
        top_2_concepts = tuple(idx2name[idx] for idx in indices)
        if top_2_concepts not in sentences:
            sentences[top_2_concepts] = []
        sentences[top_2_concepts].append(row['sentence'])
    except:
        continue

print(sentences.keys())
sentences[('Religion', 'Race')]

dict_keys([('Religion', 'Gender'), ('Religion', 'Race'), ('Race', 'Religion')])


['they are going to figure out a way to kike bitcoin',
 'the labour party has <number> mps in prison for paedophilia <number> mps and councillors on the sex registry <number> million white kids raped by paki muslims all labour voters covered up by labour councils mps and a pm',
 'do not worry uk you worked hard cannot afford carehome fees we take your house oh look in london another muzzie family in a nice flat good on you where the tex goes its a shocker',
 'dad should have told the muzrat whore to fuck off and went in anyway',
 'fanbloodytastic by time these leeches are held to account especially the muzzie scum taking <number> wives breeding kids like rabbits in order to increase welfare payments potus ur simply the best 👍 👏 👏 👏 👏',
 'meet the british nigger bitch princess criminal trash family 🤣',
 'fuck her she nothing but a kike that will play which ever side that will get her paid',
 'i am a nigger rabbi who sells bacon',
 'idiot boy trudeau his somali moslem immigration ministe

In [16]:
import numpy as np

sentences = {}
for i, row in df_cc[df_cc.label==1].iterrows():
    # print(row['concept_gradient'])
    try:
        indices = np.argsort(row['concept_gradient'])[0][-1:][::-1]
        # print(indices)
        top_2_concepts = tuple(idx2name[idx] for idx in indices)
        # top_2_concepts = tuple(sorted(top_2_concepts))
        if top_2_concepts not in sentences:
            sentences[top_2_concepts] = []
        sentences[top_2_concepts].append(row['sentence'])
    except:
        continue

print(sentences.keys())
sentences[('Race',)]
# sentences[('Religion',)]


dict_keys([('Religion',), ('Race',)])


['lets hope so gas the lot including the nigger',
 'ovomits a nigger and a half nigger',
 'match <number> a nigger wins a championship match <number> a nigger wins the battle royal now <number> niggers are singing fuck this whole thing im gonna play gta5 fuck wrestlemania 😡',
 'a mentally ill low iq nigger goes full retard after being out on a pedestal and the world is shocked',
 'how can you get rid of one nigger and replace it with another nigger x and y was much better no nigero in sight',
 'thanks to retard attempting poetry nigger rick ross filming his blasphemous and expletive laden video in and around the church of saint evstratios in vourvourou thira greek orthodox now understand what i mean when i state niggers are shit',
 'do not worry negress q speaks and we will show the nigger rigging a marcus garvey boat ticket is in your future',
 'as evidenced by the fact that they do not attack nigger toxic masculinity like rappers ball players with the same ferocity with which they at

In [26]:
df_mc[df_mc.label==1].shape

(161, 8)

In [18]:
for label_tuple, sentence_list in sentences.items():
    label = label_tuple[0] 
    filename = f"{label}_sentences.txt"
    with open(filename, 'w', encoding='utf-8') as file:
        for sentence in sentence_list:
            file.write(sentence + '\n')
    print(f"Sentences for category '{label}' have been written to {filename}")


Sentences for category 'Religion' have been written to Religion_sentences.txt
Sentences for category 'Race' have been written to Race_sentences.txt
